In [140]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [141]:
def load_mnist(dataset="training", path="."):
    if dataset is "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")
    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = keras.utils.to_categorical(np.fromfile(flbl, dtype=np.int8), 10)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        #img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols).astype('float32') / 255
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows*cols).astype('float32') / 255
   
    return img, lbl

In [153]:
def simple_dnn(batch_size, epochs):
    np.random.seed(7)
    data_path = os.path.join('.', 'data')
    x_train, y_train = load_mnist(path=data_path)
    x_test, y_test = load_mnist(dataset='testing', path=data_path)
    
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    #model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [154]:
simple_dnn(128, 10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s - loss: 0.4128 - acc: 0.8790 - val_loss: 0.1831 - val_acc: 0.9447
Epoch 2/10
60000/60000 [==============================] - 3s - loss: 0.1804 - acc: 0.9469 - val_loss: 0.1273 - val_acc: 0.9615
Epoch 3/10
60000/60000 [==============================] - 3s - loss: 0.1304 - acc: 0.9607 - val_loss: 0.0956 - val_acc: 0.9701
Epoch 4/10
60000/60000 [==============================] - 3s - loss: 0.1027 - acc: 0.9691 - val_loss: 0.0840 - val_acc: 0.9733
Epoch 5/10
60000/60000 [==============================] - 3s - loss: 0.0871 - acc: 0.9740 - val_loss: 0.0766 - val_acc: 0.9750
Epoch 6/10
60000/60000 [==============================] - 3s - loss: 0.0741 - acc: 0.9775 - val_loss: 0.0689 - val_acc: 0.9780
Epoch 7/10
60000/60000 [==============================] - 3s - loss: 0.0635 - acc: 0.9809 - val_loss: 0.0673 - val_acc: 0.9784
Epoch 8/10
60000/60000 [==============================] - 3s 

In [155]:
def cnn_dnn(batch_size, epochs):
    np.random.seed(7)
    data_path = os.path.join('.', 'data')
    x_train, y_train = load_mnist(path=data_path)
    x_test, y_test = load_mnist(dataset='testing', path=data_path)
    
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [156]:
cnn_dnn(128, 5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 7s - loss: 0.2365 - acc: 0.9300 - val_loss: 0.0676 - val_acc: 0.9790
Epoch 2/5
60000/60000 [==============================] - 6s - loss: 0.0744 - acc: 0.9771 - val_loss: 0.0442 - val_acc: 0.9866
Epoch 3/5
60000/60000 [==============================] - 6s - loss: 0.0526 - acc: 0.9837 - val_loss: 0.0367 - val_acc: 0.9874
Epoch 4/5
60000/60000 [==============================] - 6s - loss: 0.0431 - acc: 0.9865 - val_loss: 0.0333 - val_acc: 0.9901
Epoch 5/5
60000/60000 [==============================] - 6s - loss: 0.0332 - acc: 0.9895 - val_loss: 0.0314 - val_acc: 0.9903
Test loss: 0.0313963937878
Test accuracy: 0.9903


In [166]:
def cnn_deeper(batch_size, epochs):
    np.random.seed(7)
    data_path = os.path.join('.', 'data')
    x_train, y_train = load_mnist(path=data_path)
    x_test, y_test = load_mnist(dataset='testing', path=data_path)
    
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [167]:
cnn_deeper(128,5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.2541 - acc: 0.9200 - val_loss: 0.0547 - val_acc: 0.9831
Epoch 2/5
60000/60000 [==============================] - 7s - loss: 0.0691 - acc: 0.9791 - val_loss: 0.0350 - val_acc: 0.9881
Epoch 3/5
60000/60000 [==============================] - 8s - loss: 0.0518 - acc: 0.9841 - val_loss: 0.0273 - val_acc: 0.9914
Epoch 4/5
60000/60000 [==============================] - 8s - loss: 0.0400 - acc: 0.9876 - val_loss: 0.0278 - val_acc: 0.9914
Epoch 5/5
60000/60000 [==============================] - 8s - loss: 0.0330 - acc: 0.9897 - val_loss: 0.0224 - val_acc: 0.9922
Test loss: 0.0224061819654
Test accuracy: 0.9922
